In [ ]:
from github import Github
from github import Auth
from dotenv import dotenv_values
import pandas as pd
import sys
import subprocess
sys.path.append('../../')
from utils.constants import DATA_DOWNLOAD_DATE
from utils.dataloader import get_repos

config = dotenv_values("../../../.env")
auth = Auth.Token(config['GITHUB_TOKEN'])
gh = Github(auth=auth)

repositories = get_repos()

In [ ]:
for index, row in repositories.iterrows():
    if row['host'] == 'Github':
        repo = gh.get_repo(row['repo'])
        # get the commit of the last commit before the data download date, download the repo at that commit
        commits = repo.get_commits(until=DATA_DOWNLOAD_DATE)
        # get the last commit before the data download date
        if commits.totalCount > 0:
            commit = commits[0]
            print(f"Downloading {row['repo']} at commit {commit.sha}...")
            # clone the repository at that commit
            subprocess.run(['git', 'clone', repo.clone_url, f"../../../data/github/{row['repo'].replace('/', '_')}"])
            # checkout the commit
            subprocess.run(['git', 'checkout', commit.sha], cwd=f"../../../data/github/{row['repo'].replace('/', '_')}")

Cloning into '../../data/github/soot-oss_soot'...
Note: switching to '36269c759a2bfc99ed7cafa2bd1a1389af24581c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 36269c759a2 Merge pull request #2166 from MarcMil/opt


M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/GenHelloWorld.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/RunLiveAnalysis.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/RunVeryBusyAnalysis.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/analyses/FlowAnalysisTemplate.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/analyses/SimpleVeryBusyExpressions.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/analyses/VeryBusyExpressions.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/annotations/TagBusyExpressions.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/flowsets/ValueArraySparseSet.java
M	tutorial/guide/examples/analysis_framework/src/dk/brics/soot/transformations/VeryBusyExpsTagger.java
M	tutorial/guide/examples/analysis_framework/src/testers/LiveVarsClass.java
M	tutorial/guide/examples/analysis_framework/src/testers/VeryBusyClass.ja

Cloning into '../../data/github/soot-oss_SootUp'...
Note: switching to '980b110574165b0ca68060d0375324de7b322066'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 980b110574 Merge pull request #1286 from soot-oss/756-make-jimpleanalysisinputlocation-compatible-with-javaview


Cloning into '../../data/github/wala_WALA'...
Note: switching to '9336758012cc7565a524ecae8067a86a9959d37b'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 933675801 Fix issue 1515 (#1516)


Cloning into '../../data/github/secure-software-engineering_FlowDroid'...
Note: switching to '70aa5b3f938726a8dd6685eae09c84196432c6d6'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 70aa5b3f Merge pull request #820 from secure-software-engineering/dependabot/maven/soot-infoflow/org.springframework-spring-web-6.1.14


...

Cloning into '../../data/github/phpstan_phpstan'...
Note: switching to 'b42dc58e64b1730f94cbf5421f0335e7d85929f5'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at b42dc58e6 Update BACKERS.md


Cloning into '../../data/github/phpstan_phpstan-src'...
Note: switching to 'cfa029941414a746b3d74de90b8acebd5c9ed3a7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at cfa029941 Fixed false-positive with by-ref parameters and array-modifying functions


Cloning into '../../data/github/security-code-scan_security-code-scan'...
Note: switching to '815046d24e49cec43469f4852d19562a916541eb'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 815046d Update Directory.Build.props


Cloning into '../../data/github/pmd_pmd'...
Note: switching to '0d0c0a2e16130cdaa6e54aa4b951c44073416b30'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 0d0c0a2e16 Bump PMD from 7.13.0 to 7.14.0 (#5775)


Cloning into '../../data/github/spotbugs_spotbugs'...
Note: switching to 'da3610b7de3828bcca77083fbe14991448bf5f89'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at da3610b7d add `CWO_CLOSED_WITHOUT_OPENED` bugtype to `FindUnreleasedLock` detector (#3385)


In [ ]:
# loc count
for index, row in repositories.iterrows():
    if row['host'] == 'Github':
        repo_path = f"../../../data/github/{row['repo'].replace('/', '_')}"
        subprocess.run(['cloc', '.', '--csv', '--out', f'../../../results/csv/general/loc/{row["repo"].replace("/", "_")}.csv'], cwd=repo_path)

    4007 text files.
    3810 unique files.                                          
     331 files ignored.
mkdir /Users/ericzhang/Desktop/Code/Summer-Research/MSR-Static-Analysis/data/github/soot-oss_soot/../../../results/csv/loc/
Wrote ../../../results/csv/loc/soot-oss_soot.csv
    2490 text files.
    2109 unique files.                                          
    1175 files ignored.
Wrote ../../../results/csv/loc/soot-oss_SootUp.csv
    2516 text files.
    2450 unique files.                                          
      87 files ignored.
Wrote ../../../results/csv/loc/wala_WALA.csv
    1456 text files.
    1394 unique files.                                          
      91 files ignored.
Wrote ../../../results/csv/loc/secure-software-engineering_FlowDroid.csv
    1544 text files.
    1470 unique files.                                          
      97 files ignored.
Wrote ../../../results/csv/loc/secure-software-engineering_phasar.csv
    3035 text files.
    2951 unique f